# Risk Management and Metrics Framework

This notebook demonstrates comprehensive risk management capabilities from the `riskmetrics.py` module, covering VaR methodologies, credit risk analysis, and regulatory capital calculations.

## Features Covered:
- Value at Risk (Historical Simulation, Parametric, Monte Carlo)
- Expected Shortfall (Conditional VaR)
- Counterparty Credit Risk (PFE, EE, EEPE)
- Credit Risk Modeling (PD, LGD, EAD)
- Market Risk Analytics
- Regulatory Capital (Basel III)

In [ ]:
# Import necessary libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Import our riskmetrics module
from riskmetrics import (
    RiskConfig,
    VaRCalculator,
    CreditRiskAnalyzer,
    CounterpartyRiskCalculator,
    MarketRiskAnalyzer,
    RegulatoryCapitalCalculator,
    ComprehensiveRiskFramework
)

print("✅ Successfully imported risk management components")
print("⚠️ Ready to analyze market risk, credit risk, and regulatory capital")

## 1. Configuration and Setup

Initialize the risk management framework with appropriate parameters.

In [ ]:
# Initialize risk configuration
config = RiskConfig(
    confidence_level=0.95,      # 95% confidence level
    holding_period=1,           # 1-day holding period
    simulation_count=10000,     # Monte Carlo simulations
    lookback_period=252         # 1 year lookback for historical data
)

# Create the comprehensive risk framework
risk_framework = ComprehensiveRiskFramework(config)

print(f"Risk Management Framework Initialized:")
print(f"• Confidence Level: {config.confidence_level:.1%}")
print(f"• Holding Period: {config.holding_period} day(s)")
print(f"• Monte Carlo Simulations: {config.simulation_count:,}")
print(f"• Historical Lookback: {config.lookback_period} days")
print(f"• Default LGD: {config.default_lgd:.1%}")
print(f"• Risk-Free Rate: {config.risk_free_rate:.2%}")

## 2. Generate Sample Financial Data

Create realistic sample data for risk analysis demonstrations.

In [ ]:
# Generate sample portfolio returns data
def generate_sample_returns(n_assets=5, n_periods=1000, correlation_level=0.3):
    """
    Generate correlated asset returns for portfolio analysis.
    """
    # Create correlation matrix
    correlation_matrix = np.full((n_assets, n_assets), correlation_level)
    np.fill_diagonal(correlation_matrix, 1.0)
    
    # Generate correlated returns
    independent_returns = np.random.multivariate_normal(
        mean=np.zeros(n_assets),
        cov=correlation_matrix,
        size=n_periods
    )
    
    # Scale returns to realistic levels
    mean_returns = np.array([0.0008, 0.0006, 0.0010, 0.0005, 0.0012])  # Daily means
    volatilities = np.array([0.015, 0.020, 0.025, 0.012, 0.030])       # Daily volatilities
    
    scaled_returns = independent_returns * volatilities + mean_returns
    
    # Create DataFrame
    asset_names = ['Equity_US', 'Equity_EU', 'Equity_EM', 'Bonds_Gov', 'Commodities']
    dates = pd.date_range(start='2020-01-01', periods=n_periods, freq='D')
    
    returns_df = pd.DataFrame(scaled_returns, columns=asset_names, index=dates)
    
    return returns_df

# Generate sample credit data
def generate_sample_credit_data(n_obligors=100):
    """
    Generate sample credit portfolio data.
    """
    # Rating mapping
    ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC']
    rating_weights = [0.05, 0.10, 0.20, 0.30, 0.20, 0.10, 0.05]
    
    # PD mapping by rating
    pd_mapping = {
        'AAA': 0.0002, 'AA': 0.0005, 'A': 0.0015, 'BBB': 0.0040,
        'BB': 0.0120, 'B': 0.0350, 'CCC': 0.1000
    }
    
    credit_data = []
    for i in range(n_obligors):
        rating = np.random.choice(ratings, p=rating_weights)
        exposure = np.random.lognormal(15, 1)  # Log-normal exposure
        maturity = np.random.uniform(0.5, 10)   # 0.5 to 10 years
        
        credit_data.append({
            'obligor_id': f'OB_{i+1:03d}',
            'rating': rating,
            'pd': pd_mapping[rating],
            'exposure': exposure,
            'lgd': np.random.uniform(0.3, 0.7),  # 30-70% LGD
            'maturity': maturity,
            'sector': np.random.choice(['Financial', 'Industrial', 'Technology', 'Healthcare', 'Energy'])
        })
    
    return pd.DataFrame(credit_data)

# Generate the sample data
returns_data = generate_sample_returns()
credit_data = generate_sample_credit_data()

print(f"📊 Generated Sample Data:")
print(f"• Portfolio Returns: {returns_data.shape[0]} days × {returns_data.shape[1]} assets")
print(f"• Credit Portfolio: {len(credit_data)} obligors")
print(f"• Date Range: {returns_data.index[0].date()} to {returns_data.index[-1].date()}")

# Display sample statistics
print(f"\n📈 Portfolio Statistics:")
print(returns_data.describe())

## 3. Value at Risk (VaR) Analysis

Calculate VaR using multiple methodologies and compare results.

In [ ]:
# Initialize VaR calculator
var_calculator = VaRCalculator(config)

# Portfolio weights (equal weight for demonstration)
portfolio_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
portfolio_value = 10_000_000  # $10 million portfolio

# Calculate portfolio returns
portfolio_returns = (returns_data * portfolio_weights).sum(axis=1)

print(f"📊 Value at Risk Analysis:")
print("=" * 50)

# 1. Historical Simulation VaR
historical_var = var_calculator.historical_simulation_var(
    portfolio_returns, confidence_level=0.95
)

print(f"📈 Historical Simulation VaR:")
print(f"• VaR (95%): {historical_var['var']:.4f} ({historical_var['var']*portfolio_value:,.0f} USD)")
print(f"• Expected Shortfall: {historical_var['expected_shortfall']:.4f}")
print(f"• Worst Loss: {historical_var['worst_loss']:.4f}")
print(f"• Number of Exceedances: {historical_var['exceedances']}")

# 2. Parametric VaR
parametric_var = var_calculator.parametric_var(
    portfolio_returns, confidence_level=0.95
)

print(f"\n📊 Parametric VaR:")
print(f"• VaR (95%): {parametric_var['var']:.4f} ({parametric_var['var']*portfolio_value:,.0f} USD)")
print(f"• Expected Shortfall: {parametric_var['expected_shortfall']:.4f}")
print(f"• Portfolio Mean: {parametric_var['mean']:.4f}")
print(f"• Portfolio Volatility: {parametric_var['std']:.4f}")

# 3. Monte Carlo VaR
monte_carlo_var = var_calculator.monte_carlo_var(
    portfolio_returns, confidence_level=0.95, simulations=10000
)

print(f"\n🎲 Monte Carlo VaR:")
print(f"• VaR (95%): {monte_carlo_var['var']:.4f} ({monte_carlo_var['var']*portfolio_value:,.0f} USD)")
print(f"• Expected Shortfall: {monte_carlo_var['expected_shortfall']:.4f}")
print(f"• Simulation Std Error: {monte_carlo_var['std_error']:.4f}")
print(f"• Number of Simulations: {monte_carlo_var['simulations']:,}")

# VaR Backtesting
backtest_results = var_calculator.var_backtesting(
    portfolio_returns, historical_var['var'], confidence_level=0.95
)

print(f"\n🔍 VaR Backtesting (Historical VaR):")
print(f"• Actual Exceedances: {backtest_results['actual_exceedances']}")
print(f"• Expected Exceedances: {backtest_results['expected_exceedances']:.1f}")
print(f"• Kupiec p-value: {backtest_results['kupiec_pvalue']:.4f}")
print(f"• Model Adequacy: {'PASS' if backtest_results['kupiec_pvalue'] > 0.05 else 'FAIL'}")
print(f"• Exception Rate: {backtest_results['exception_rate']:.2%}")

# Multi-period VaR
multi_period_var = var_calculator.multi_period_var(
    portfolio_returns, confidence_level=0.95, holding_periods=[1, 5, 10, 22]
)

print(f"\n📅 Multi-Period VaR Analysis:")
for period, var_value in multi_period_var.items():
    print(f"• {period}-day VaR: {var_value:.4f} ({var_value*portfolio_value:,.0f} USD)")

## 4. Credit Risk Analysis

Analyze credit risk including PD modeling, migration analysis, and portfolio credit risk.

In [ ]:
# Initialize credit risk analyzer
credit_analyzer = CreditRiskAnalyzer(config)

print(f"💳 Credit Risk Analysis:")
print("=" * 40)

# 1. PD from Bond Prices (simulate bond data)
bond_prices = np.array([98.5, 97.2, 95.8, 92.1, 88.5])  # Bond prices
face_values = np.array([100, 100, 100, 100, 100])       # Face values
maturities = np.array([1, 2, 3, 5, 7])                  # Years to maturity
coupon_rates = np.array([0.03, 0.035, 0.04, 0.045, 0.05])  # Coupon rates

bond_pd_analysis = credit_analyzer.pd_from_bond_prices(
    bond_prices, face_values, coupon_rates, maturities, risk_free_rate=0.02
)

print(f"📊 PD from Bond Prices:")
for i, (maturity, pd, spread) in enumerate(zip(maturities, 
                                              bond_pd_analysis['implied_pds'],
                                              bond_pd_analysis['credit_spreads'])):
    print(f"• {maturity}Y Bond: PD = {pd:.2%}, Spread = {spread:.0f} bps")

print(f"\nAverage Implied PD: {np.mean(bond_pd_analysis['implied_pds']):.2%}")

# 2. Credit Spread Analysis
market_yields = np.array([0.025, 0.030, 0.035, 0.042, 0.048])  # Market yields
risk_free_yields = np.array([0.020, 0.022, 0.024, 0.026, 0.028])  # Risk-free yields

spread_analysis = credit_analyzer.credit_spread_analysis(
    market_yields, risk_free_yields, maturities
)

print(f"\n📈 Credit Spread Analysis:")
print(f"• Credit Spreads: {[f'{s:.0f} bps' for s in spread_analysis['credit_spreads']]}")
print(f"• Average Spread: {np.mean(spread_analysis['credit_spreads']):.0f} bps")
print(f"• Spread Volatility: {np.std(spread_analysis['credit_spreads']):.0f} bps")
print(f"• Term Structure Slope: {spread_analysis['term_structure_slope']:.2f} bps/year")

# 3. Transition Matrix Analysis
# Sample historical rating data
historical_ratings = pd.DataFrame({
    'obligor_id': [f'OB_{i:03d}' for i in range(1, 101)],
    'rating_t0': np.random.choice(['AAA', 'AA', 'A', 'BBB', 'BB', 'B'], 100),
    'rating_t1': np.random.choice(['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'D'], 100)
})

transition_analysis = credit_analyzer.rating_transition_analysis(
    historical_ratings, from_col='rating_t0', to_col='rating_t1'
)

print(f"\n🔄 Rating Transition Matrix:")
transition_matrix = transition_analysis['transition_matrix']
print(transition_matrix.round(3))

print(f"\nDefault Rates by Rating:")
for rating in transition_matrix.index:
    if 'D' in transition_matrix.columns:
        default_rate = transition_matrix.loc[rating, 'D']
        print(f"• {rating}: {default_rate:.2%}")

# 4. Portfolio Credit Risk
portfolio_credit_risk = credit_analyzer.portfolio_credit_risk(
    credit_data, correlation_factor=0.15
)

print(f"\n💼 Portfolio Credit Risk Analysis:")
print(f"• Total Exposure: ${portfolio_credit_risk['total_exposure']:,.0f}")
print(f"• Expected Loss: ${portfolio_credit_risk['expected_loss']:,.0f}")
print(f"• Unexpected Loss: ${portfolio_credit_risk['unexpected_loss']:,.0f}")
print(f"• Credit VaR (99.9%): ${portfolio_credit_risk['credit_var']:,.0f}")
print(f"• Portfolio Default Rate: {portfolio_credit_risk['portfolio_pd']:.3%}")
print(f"• Concentration Risk: {portfolio_credit_risk['concentration_measure']:.3f}")

# Sector concentration
sector_exposure = credit_data.groupby('sector')['exposure'].sum().sort_values(ascending=False)
print(f"\n🏭 Sector Concentration:")
for sector, exposure in sector_exposure.items():
    percentage = exposure / sector_exposure.sum() * 100
    print(f"• {sector}: ${exposure:,.0f} ({percentage:.1f}%)")

## 5. Counterparty Risk Analysis

Calculate counterparty credit risk metrics including PFE, EE, and EEPE.

In [ ]:
# Initialize counterparty risk calculator
counterparty_calc = CounterpartyRiskCalculator(config)

print(f"🤝 Counterparty Credit Risk Analysis:")
print("=" * 45)

# Generate sample derivative portfolio
def generate_derivative_portfolio():
    """Generate sample derivative exposures."""
    time_points = np.linspace(0, 5, 21)  # 5 years, quarterly points
    n_scenarios = 1000
    
    # Simulate interest rate swaps exposures
    exposures = []
    for i, t in enumerate(time_points):
        # Exposure decreases over time but with volatility
        base_exposure = 1000000 * np.exp(-0.1 * t)  # Decreasing notional effect
        scenario_exposures = np.maximum(0, 
            base_exposure + np.random.normal(0, base_exposure * 0.3, n_scenarios)
        )
        exposures.append(scenario_exposures)
    
    return np.array(exposures), time_points

exposures, time_points = generate_derivative_portfolio()

# Calculate PFE
pfe_analysis = counterparty_calc.calculate_potential_future_exposure(
    exposures, time_points, confidence_levels=[0.95, 0.975, 0.99]
)

print(f"📊 Potential Future Exposure (PFE):")
for confidence, pfe_values in pfe_analysis['pfe_by_confidence'].items():
    max_pfe = np.max(pfe_values)
    max_time = time_points[np.argmax(pfe_values)]
    print(f"• PFE {confidence:.1%}: ${max_pfe:,.0f} (peak at {max_time:.1f} years)")

print(f"\nPFE Profile Statistics:")
print(f"• Average PFE (95%): ${np.mean(pfe_analysis['pfe_by_confidence'][0.95]):,.0f}")
print(f"• Peak PFE Time: {time_points[np.argmax(pfe_analysis['pfe_by_confidence'][0.95])]:.1f} years")

# Calculate Expected Exposure (EE)
ee_analysis = counterparty_calc.calculate_expected_exposure(
    exposures, time_points
)

print(f"\n📈 Expected Exposure (EE):")
print(f"• Peak EE: ${np.max(ee_analysis['expected_exposure']):,.0f}")
print(f"• Average EE: ${np.mean(ee_analysis['expected_exposure']):,.0f}")
print(f"• EE at 1 year: ${ee_analysis['expected_exposure'][4]:,.0f}")  # Assuming quarterly points

# Calculate EEPE
eepe_analysis = counterparty_calc.calculate_eepe(
    exposures, time_points, maturity=5.0
)

print(f"\n⚡ Effective Expected Positive Exposure (EEPE):")
print(f"• EEPE: ${eepe_analysis['eepe']:,.0f}")
print(f"• Time-weighted Average EE: ${eepe_analysis['time_weighted_ee']:,.0f}")
print(f"• Maturity: {eepe_analysis['maturity']} years")

# Calculate LER (Loan Equivalent Risk)
counterparty_pd = 0.02  # 2% counterparty PD
counterparty_lgd = 0.6  # 60% LGD

ler_analysis = counterparty_calc.calculate_loan_equivalent_risk(
    exposures, time_points, counterparty_pd, counterparty_lgd
)

print(f"\n💳 Loan Equivalent Risk (LER):")
print(f"• LER: ${ler_analysis['ler']:,.0f}")
print(f"• Credit Risk Capital: ${ler_analysis['credit_risk_capital']:,.0f}")
print(f"• Counterparty PD: {counterparty_pd:.2%}")
print(f"• Counterparty LGD: {counterparty_lgd:.1%}")

# CVA Calculation
survival_probabilities = [(1 - counterparty_pd) ** t for t in time_points]
discount_factors = [np.exp(-config.risk_free_rate * t) for t in time_points]

cva_analysis = counterparty_calc.calculate_cva(
    ee_analysis['expected_exposure'], 
    survival_probabilities, 
    discount_factors, 
    counterparty_lgd
)

print(f"\n💰 Credit Valuation Adjustment (CVA):")
print(f"• CVA: ${cva_analysis['cva']:,.0f}")
print(f"• CVA as % of Exposure: {cva_analysis['cva_percentage']:.3%}")
print(f"• Expected Loss: ${cva_analysis['expected_loss']:,.0f}")

# Wrong-way risk analysis
correlation_exposure_default = -0.3  # Negative correlation (wrong-way risk)

wwr_analysis = counterparty_calc.wrong_way_risk_analysis(
    exposures, counterparty_pd, correlation_exposure_default
)

print(f"\n⚠️ Wrong-Way Risk Analysis:")
print(f"• WWR Adjustment Factor: {wwr_analysis['wwr_adjustment']:.3f}")
print(f"• Adjusted CVA: ${wwr_analysis['adjusted_cva']:,.0f}")
print(f"• CVA Increase: ${wwr_analysis['cva_increase']:,.0f}")
print(f"• Correlation (Exposure-Default): {correlation_exposure_default:.2f}")

## 6. Market Risk Analytics

Comprehensive market risk analysis including stress testing and scenario analysis.

In [ ]:
# Initialize market risk analyzer
market_analyzer = MarketRiskAnalyzer(config)

print(f"📊 Market Risk Analytics:")
print("=" * 35)

# 1. Stress Testing
stress_scenarios = {
    'market_crash': {
        'equity_shock': -0.30,    # 30% equity decline
        'rates_shock': -0.02,     # 200 bps rate decline
        'fx_shock': 0.15,         # 15% USD strengthening
        'credit_shock': 0.005     # 500 bps credit spread widening
    },
    'inflation_surge': {
        'equity_shock': -0.15,
        'rates_shock': 0.03,      # 300 bps rate increase
        'fx_shock': -0.10,
        'credit_shock': 0.002
    },
    'moderate_stress': {
        'equity_shock': -0.10,
        'rates_shock': 0.01,
        'fx_shock': 0.05,
        'credit_shock': 0.001
    }
}

stress_results = market_analyzer.stress_testing(
    returns_data, portfolio_weights, stress_scenarios, portfolio_value
)

print(f"🚨 Stress Testing Results:")
for scenario_name, result in stress_results.items():
    print(f"\n• {scenario_name.replace('_', ' ').title()}:")
    print(f"  Portfolio Loss: ${result['portfolio_loss']:,.0f}")
    print(f"  Loss Percentage: {result['loss_percentage']:.2%}")
    print(f"  Worst Asset: {result['worst_asset']} ({result['worst_asset_loss']:.2%})")
    print(f"  Risk Contribution: {result['risk_contribution']}")

# 2. Scenario Analysis
economic_scenarios = {
    'recession': {'growth': -0.02, 'inflation': 0.01, 'rates': -0.01},
    'expansion': {'growth': 0.04, 'inflation': 0.03, 'rates': 0.02},
    'stagflation': {'growth': -0.005, 'inflation': 0.06, 'rates': 0.03}
}

scenario_results = market_analyzer.scenario_analysis(
    returns_data, portfolio_weights, economic_scenarios, portfolio_value
)

print(f"\n🌍 Economic Scenario Analysis:")
for scenario, result in scenario_results.items():
    print(f"\n• {scenario.title()} Scenario:")
    print(f"  Expected Return: {result['expected_return']:.2%}")
    print(f"  Portfolio Value Impact: ${result['value_impact']:,.0f}")
    print(f"  Risk Level: {result['risk_level']}")
    print(f"  Probability: {result.get('probability', 'N/A')}")

# 3. Sensitivity Analysis
risk_factors = ['interest_rates', 'equity_markets', 'credit_spreads', 'fx_rates']
factor_shocks = np.linspace(-0.05, 0.05, 11)  # -5% to +5% shocks

sensitivity_results = market_analyzer.sensitivity_analysis(
    returns_data, portfolio_weights, risk_factors, factor_shocks, portfolio_value
)

print(f"\n📈 Sensitivity Analysis:")
for factor, sensitivities in sensitivity_results.items():
    max_loss = min(sensitivities['portfolio_impacts'])
    max_gain = max(sensitivities['portfolio_impacts'])
    print(f"• {factor.replace('_', ' ').title()}:")
    print(f"  Max Loss (5% shock): ${max_loss:,.0f}")
    print(f"  Max Gain (5% shock): ${max_gain:,.0f}")
    print(f"  Sensitivity: ${sensitivities['sensitivity']:,.0f} per 1% shock")

# 4. Component VaR
component_var = market_analyzer.component_var(
    returns_data, portfolio_weights, confidence_level=0.95
)

print(f"\n🧩 Component VaR Analysis:")
print(f"Total Portfolio VaR: {component_var['portfolio_var']:.4f}")
print(f"\nComponent VaR by Asset:")
for asset, cvar in component_var['component_vars'].items():
    contribution = cvar / component_var['portfolio_var'] * 100
    print(f"• {asset}: {cvar:.4f} ({contribution:.1f}%)")

print(f"\nMarginal VaR:")
for asset, mvar in component_var['marginal_vars'].items():
    print(f"• {asset}: {mvar:.4f}")

# 5. Risk Attribution
risk_attribution = market_analyzer.risk_attribution_analysis(
    returns_data, portfolio_weights
)

print(f"\n🎯 Risk Attribution:")
print(f"Portfolio Volatility: {risk_attribution['portfolio_volatility']:.3%}")
print(f"\nVolatility Contribution:")
for asset, contrib in risk_attribution['volatility_contributions'].items():
    print(f"• {asset}: {contrib:.4f} ({contrib/risk_attribution['portfolio_volatility']*100:.1f}%)")

print(f"\nDiversification Benefit: {risk_attribution['diversification_benefit']:.4f}")
print(f"Diversification Ratio: {risk_attribution['diversification_ratio']:.3f}")

## 7. Regulatory Capital Calculations

Calculate regulatory capital requirements under Basel III framework.

In [ ]:
# Initialize regulatory capital calculator
capital_calc = RegulatoryCapitalCalculator(config)

print(f"🏛️ Regulatory Capital Analysis (Basel III):")
print("=" * 50)

# 1. Market Risk Capital (Standardized Approach)
market_positions = {
    'equity_positions': 5000000,      # $5M equity exposure
    'interest_rate_positions': 8000000,  # $8M IR exposure
    'fx_positions': 2000000,          # $2M FX exposure
    'commodity_positions': 1000000    # $1M commodity exposure
}

market_risk_capital = capital_calc.market_risk_capital(
    market_positions, approach='standardized'
)

print(f"📊 Market Risk Capital:")
print(f"• Equity Risk Capital: ${market_risk_capital['equity_capital']:,.0f}")
print(f"• Interest Rate Risk Capital: ${market_risk_capital['interest_rate_capital']:,.0f}")
print(f"• FX Risk Capital: ${market_risk_capital['fx_capital']:,.0f}")
print(f"• Commodity Risk Capital: ${market_risk_capital['commodity_capital']:,.0f}")
print(f"• Total Market Risk Capital: ${market_risk_capital['total_capital']:,.0f}")
print(f"• Capital Ratio: {market_risk_capital['capital_ratio']:.2%}")

# 2. Credit Risk Capital (Standardized Approach)
credit_positions = {
    'corporate_aaa': {'exposure': 10000000, 'risk_weight': 0.20},
    'corporate_aa': {'exposure': 15000000, 'risk_weight': 0.50},
    'corporate_a': {'exposure': 20000000, 'risk_weight': 0.50},
    'corporate_bbb': {'exposure': 25000000, 'risk_weight': 1.00},
    'corporate_bb': {'exposure': 8000000, 'risk_weight': 1.50},
    'residential_mortgage': {'exposure': 30000000, 'risk_weight': 0.35},
    'government': {'exposure': 5000000, 'risk_weight': 0.00}
}

credit_risk_capital = capital_calc.credit_risk_capital(
    credit_positions, approach='standardized'
)

print(f"\n💳 Credit Risk Capital:")
print(f"• Total Exposure: ${credit_risk_capital['total_exposure']:,.0f}")
print(f"• Risk-Weighted Assets: ${credit_risk_capital['rwa']:,.0f}")
print(f"• Credit Risk Capital: ${credit_risk_capital['capital_requirement']:,.0f}")
print(f"• Average Risk Weight: {credit_risk_capital['average_risk_weight']:.1%}")

print(f"\nBreakdown by Asset Class:")
for asset_class, details in credit_risk_capital['breakdown'].items():
    print(f"• {asset_class.replace('_', ' ').title()}: RWA ${details['rwa']:,.0f}, Capital ${details['capital']:,.0f}")

# 3. Operational Risk Capital
business_indicators = {
    'interest_income': 50000000,
    'interest_expense': 20000000,
    'fee_income': 30000000,
    'trading_income': 15000000,
    'other_income': 10000000
}

operational_risk_capital = capital_calc.operational_risk_capital(
    business_indicators, approach='standardized'
)

print(f"\n⚙️ Operational Risk Capital:")
print(f"• Business Indicator Component: ${operational_risk_capital['business_indicator']:,.0f}")
print(f"• Internal Loss Multiplier: {operational_risk_capital['loss_multiplier']:.3f}")
print(f"• Operational Risk Capital: ${operational_risk_capital['capital_requirement']:,.0f}")
print(f"• Capital as % of BI: {operational_risk_capital['capital_percentage']:.2%}")

# 4. Total Capital Requirements
total_capital_analysis = capital_calc.total_capital_requirements(
    market_risk_capital['total_capital'],
    credit_risk_capital['capital_requirement'],
    operational_risk_capital['capital_requirement']
)

print(f"\n🏛️ Total Capital Requirements:")
print(f"• Market Risk Capital: ${total_capital_analysis['market_risk']:,.0f}")
print(f"• Credit Risk Capital: ${total_capital_analysis['credit_risk']:,.0f}")
print(f"• Operational Risk Capital: ${total_capital_analysis['operational_risk']:,.0f}")
print(f"• Total Pillar 1 Capital: ${total_capital_analysis['total_pillar1']:,.0f}")
print(f"• Capital Conservation Buffer: ${total_capital_analysis['conservation_buffer']:,.0f}")
print(f"• Countercyclical Buffer: ${total_capital_analysis['countercyclical_buffer']:,.0f}")
print(f"• Total Capital Requirement: ${total_capital_analysis['total_requirement']:,.0f}")

# 5. Risk-Weighted Assets Summary
total_rwa = credit_risk_capital['rwa'] + (market_risk_capital['total_capital'] * 12.5) + (operational_risk_capital['capital_requirement'] * 12.5)

print(f"\n📊 Risk-Weighted Assets Summary:")
print(f"• Credit RWA: ${credit_risk_capital['rwa']:,.0f}")
print(f"• Market RWA: ${market_risk_capital['total_capital'] * 12.5:,.0f}")
print(f"• Operational RWA: ${operational_risk_capital['capital_requirement'] * 12.5:,.0f}")
print(f"• Total RWA: ${total_rwa:,.0f}")
print(f"• Capital Ratio (8%): {total_capital_analysis['total_requirement'] / total_rwa:.2%}")

# 6. LCOE (Leverage Coverage Ratio)
tier1_capital = total_capital_analysis['total_requirement'] * 1.2  # Assume Tier 1 is 20% higher
total_exposure = sum(pos['exposure'] for pos in credit_positions.values()) + sum(market_positions.values())

leverage_ratio = tier1_capital / total_exposure

print(f"\n⚖️ Leverage Ratio:")
print(f"• Tier 1 Capital: ${tier1_capital:,.0f}")
print(f"• Total Exposure: ${total_exposure:,.0f}")
print(f"• Leverage Ratio: {leverage_ratio:.2%}")
print(f"• Minimum Requirement (3%): {'PASS' if leverage_ratio >= 0.03 else 'FAIL'}")

## 8. Risk Dashboard Visualization

Create comprehensive risk visualizations and dashboards.

In [ ]:
# Create comprehensive risk dashboard
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
fig = plt.figure(figsize=(20, 16))

# 1. VaR Comparison
ax1 = plt.subplot(3, 4, 1)
var_methods = ['Historical', 'Parametric', 'Monte Carlo']
var_values = [
    historical_var['var'] * portfolio_value,
    parametric_var['var'] * portfolio_value,
    monte_carlo_var['var'] * portfolio_value
]

bars = ax1.bar(var_methods, var_values, color=['blue', 'orange', 'green'], alpha=0.7)
ax1.set_title('VaR Comparison (95%)', fontweight='bold')
ax1.set_ylabel('VaR (USD)')
ax1.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, value in zip(bars, var_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'${value:,.0f}', ha='center', va='bottom', fontsize=9)

# 2. Portfolio Returns Distribution
ax2 = plt.subplot(3, 4, 2)
ax2.hist(portfolio_returns, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(historical_var['var'], color='red', linestyle='--', linewidth=2, label=f'VaR 95%')
ax2.axvline(np.mean(portfolio_returns), color='green', linestyle='--', linewidth=2, label='Mean')
ax2.set_title('Portfolio Returns Distribution', fontweight='bold')
ax2.set_xlabel('Daily Returns')
ax2.set_ylabel('Frequency')
ax2.legend()

# 3. Credit Portfolio Composition
ax3 = plt.subplot(3, 4, 3)
rating_exposure = credit_data.groupby('rating')['exposure'].sum()
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(rating_exposure)))
wedges, texts, autotexts = ax3.pie(rating_exposure.values, labels=rating_exposure.index, 
                                  autopct='%1.1f%%', colors=colors, startangle=90)
ax3.set_title('Credit Portfolio by Rating', fontweight='bold')

# 4. PFE Profile
ax4 = plt.subplot(3, 4, 4)
ax4.plot(time_points, pfe_analysis['pfe_by_confidence'][0.95], 'b-', linewidth=2, label='PFE 95%')
ax4.plot(time_points, pfe_analysis['pfe_by_confidence'][0.99], 'r-', linewidth=2, label='PFE 99%')
ax4.plot(time_points, ee_analysis['expected_exposure'], 'g--', linewidth=2, label='Expected Exposure')
ax4.set_title('Counterparty Exposure Profile', fontweight='bold')
ax4.set_xlabel('Time (Years)')
ax4.set_ylabel('Exposure (USD)')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Stress Test Results
ax5 = plt.subplot(3, 4, 5)
scenarios = list(stress_results.keys())
losses = [result['portfolio_loss'] for result in stress_results.values()]
colors = ['red' if loss < -1000000 else 'orange' if loss < -500000 else 'yellow' for loss in losses]

bars = ax5.bar(scenarios, losses, color=colors, alpha=0.7)
ax5.set_title('Stress Test Results', fontweight='bold')
ax5.set_ylabel('Portfolio Loss (USD)')
ax5.tick_params(axis='x', rotation=45)

# 6. Component VaR
ax6 = plt.subplot(3, 4, 6)
assets = list(component_var['component_vars'].keys())
component_vars = [abs(var) for var in component_var['component_vars'].values()]

bars = ax6.barh(assets, component_vars, color='lightcoral', alpha=0.7)
ax6.set_title('Component VaR by Asset', fontweight='bold')
ax6.set_xlabel('Component VaR')

# 7. Capital Requirements
ax7 = plt.subplot(3, 4, 7)
capital_types = ['Market Risk', 'Credit Risk', 'Operational Risk']
capital_amounts = [
    market_risk_capital['total_capital'],
    credit_risk_capital['capital_requirement'],
    operational_risk_capital['capital_requirement']
]

bars = ax7.bar(capital_types, capital_amounts, color=['blue', 'green', 'orange'], alpha=0.7)
ax7.set_title('Regulatory Capital Requirements', fontweight='bold')
ax7.set_ylabel('Capital Requirement (USD)')
ax7.tick_params(axis='x', rotation=45)

# 8. Risk Attribution
ax8 = plt.subplot(3, 4, 8)
assets = list(risk_attribution['volatility_contributions'].keys())
contributions = list(risk_attribution['volatility_contributions'].values())
contributions_pct = [c/sum(contributions)*100 for c in contributions]

bars = ax8.bar(assets, contributions_pct, color='lightgreen', alpha=0.7)
ax8.set_title('Risk Contribution by Asset', fontweight='bold')
ax8.set_ylabel('Risk Contribution (%)')
ax8.tick_params(axis='x', rotation=45)

# 9. Credit Spread Term Structure
ax9 = plt.subplot(3, 4, 9)
ax9.plot(maturities, spread_analysis['credit_spreads'], 'bo-', linewidth=2, markersize=6)
ax9.set_title('Credit Spread Term Structure', fontweight='bold')
ax9.set_xlabel('Maturity (Years)')
ax9.set_ylabel('Credit Spread (bps)')
ax9.grid(True, alpha=0.3)

# 10. VaR Backtesting
ax10 = plt.subplot(3, 4, 10)
breach_dates = portfolio_returns[portfolio_returns < historical_var['var']].index[-20:]  # Last 20 breaches
if len(breach_dates) > 0:
    breach_returns = portfolio_returns[portfolio_returns < historical_var['var']][-20:]
    ax10.scatter(range(len(breach_returns)), breach_returns, color='red', alpha=0.7, s=50)
    ax10.axhline(historical_var['var'], color='blue', linestyle='--', linewidth=2, label='VaR Threshold')
    ax10.set_title('VaR Exceedances', fontweight='bold')
    ax10.set_xlabel('Breach Instance')
    ax10.set_ylabel('Return')
    ax10.legend()
else:
    ax10.text(0.5, 0.5, 'No VaR Breaches\nin Sample Period', 
              ha='center', va='center', transform=ax10.transAxes, fontsize=12)
    ax10.set_title('VaR Exceedances', fontweight='bold')

# 11. Correlation Heatmap
ax11 = plt.subplot(3, 4, 11)
correlation_matrix = returns_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
           square=True, ax=ax11, cbar_kws={'shrink': 0.8})
ax11.set_title('Asset Correlation Matrix', fontweight='bold')

# 12. Risk Metrics Summary
ax12 = plt.subplot(3, 4, 12)
ax12.axis('off')

# Create summary text
summary_text = f"""
RISK METRICS SUMMARY
═══════════════════

Portfolio VaR (95%):
${historical_var['var']*portfolio_value:,.0f}

Expected Shortfall:
${historical_var['expected_shortfall']*portfolio_value:,.0f}

Portfolio Volatility:
{risk_attribution['portfolio_volatility']:.2%}

Total Capital Req:
${total_capital_analysis['total_requirement']:,.0f}

Leverage Ratio:
{leverage_ratio:.2%}

Credit Portfolio EL:
${portfolio_credit_risk['expected_loss']:,.0f}

Max PFE (95%):
${np.max(pfe_analysis['pfe_by_confidence'][0.95]):,.0f}
"""

ax12.text(0.1, 0.9, summary_text, transform=ax12.transAxes, fontsize=10,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

plt.tight_layout()
plt.show()

print("📊 Comprehensive risk dashboard created successfully!")
print("\n🎯 Key Risk Insights:")
print(f"• Portfolio 1-day VaR represents {historical_var['var']*100:.2f}% of portfolio value")
print(f"• Credit portfolio concentration in {credit_data.groupby('sector')['exposure'].sum().idxmax()} sector")
print(f"• Counterparty exposure peaks at {time_points[np.argmax(pfe_analysis['pfe_by_confidence'][0.95])]:.1f} years")
print(f"• Total regulatory capital is {total_capital_analysis['total_requirement']/total_rwa:.1%} of RWA")

## Summary

This notebook demonstrated comprehensive risk management capabilities:

### ✅ **Risk Analysis Components:**

1. **Value at Risk (VaR)** - Historical simulation, parametric, and Monte Carlo methods
2. **Credit Risk Analysis** - PD modeling, rating transitions, portfolio credit risk
3. **Counterparty Risk** - PFE, EE, EEPE, CVA, and wrong-way risk
4. **Market Risk Analytics** - Stress testing, scenario analysis, sensitivity analysis
5. **Regulatory Capital** - Basel III market, credit, and operational risk capital
6. **Risk Attribution** - Component VaR, marginal VaR, diversification analysis

### 🔧 **Key Features:**
- **Multi-methodology Approach**: Combines historical, parametric, and simulation methods
- **Regulatory Compliance**: Basel III compliant capital calculations
- **Comprehensive Coverage**: Market, credit, operational, and counterparty risks
- **Advanced Analytics**: Stress testing, backtesting, and attribution analysis
- **Professional Visualizations**: Risk dashboards and interactive charts

### 📊 **Typical Results:**
- **VaR (95%)**: 1.5-3.0% of portfolio value for diversified portfolios
- **Expected Shortfall**: 1.5-2x VaR depending on tail risk
- **Capital Ratios**: 8-12% of risk-weighted assets under Basel III
- **Credit Losses**: 0.1-2.0% annually depending on portfolio quality

### 🎯 **Business Applications:**
- **Risk Monitoring**: Daily risk monitoring and limit management
- **Capital Planning**: Regulatory capital optimization and planning
- **Portfolio Management**: Risk-adjusted performance measurement
- **Regulatory Reporting**: Basel III and stress testing compliance
- **Strategic Decision Making**: Risk-return optimization and business planning

The framework provides institutional-grade risk management tools suitable for banks, asset managers, and financial institutions requiring comprehensive risk measurement and regulatory compliance.